In [1]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model


from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score

import os
#import chardet
import pandas as pd
from pathlib import Path

# The file path is correctly resolved.
file_path = Path('../data/Customer-Churn-Dataset1.csv').resolve()
df=pd.read_csv(file_path)
df
df_encoded = df.copy()
le = LabelEncoder()
df_encoded['Churn'] = le.fit_transform(df_encoded['Churn'])
df_encoded

df_encoded = df_encoded.drop('customerID', axis=1)
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']
X
y
X['TotalCharges'].value_counts()





TotalCharges
          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: count, Length: 6531, dtype: int64

In [2]:
X.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
numAdminTickets     0
numTechTickets      0
dtype: int64

In [3]:
df_encoded.duplicated().sum()

np.int64(17)

In [4]:
df_encoded[df_encoded.duplicated(keep=False)]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
100,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,20.20,20.2,0,0,0
542,Female,0,No,No,1,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,19.55,19.55,0,0,0
646,Male,0,No,No,1,Yes,No,DSL,No,No,...,No,No,Month-to-month,Yes,Mailed check,45.70,45.7,0,0,1
662,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,20.05,20.05,0,0,0
690,Male,0,No,No,1,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Month-to-month,Yes,Mailed check,20.45,20.45,0,0,0
964,Male,0,No,No,1,Yes,No,DSL,No,No,...,No,No,Month-to-month,Yes,Mailed check,45.70,45.7,0,0,1
976,Male,0,No,No,1,Yes,No,Fiber optic,No,No,...,No,No,Month-to-month,Yes,Electronic check,69.90,69.9,0,0,1
1243,Male,0,No,No,1,Yes,No,DSL,No,No,...,No,No,Month-to-month,No,Electronic check,45.30,45.3,0,0,1
1491,Female,0,No,No,1,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,19.55,19.55,0,0,0
1731,Female,1,No,No,1,Yes,No,Fiber optic,No,No,...,No,No,Month-to-month,Yes,Electronic check,69.60,69.6,0,0,1


In [12]:
df_encoded = df_encoded.drop_duplicates()

In [5]:
X.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
numAdminTickets       int64
numTechTickets        int64
dtype: object

In [6]:
# First, handle the TotalCharges column
# Drop duplicates and convert TotalCharges
X['TotalCharges'] = pd.to_numeric(X['TotalCharges'], errors='coerce')
X['TotalCharges'].fillna(X['TotalCharges'].mean(), inplace=True)

X.dropna(inplace=True)
X['TotalCharges']

C:\Users\Chandra\AppData\Local\Temp\ipykernel_6900\2045035441.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['TotalCharges'].fillna(X['TotalCharges'].mean(), inplace=True)


0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

In [8]:
#  Identify target and features
target_col = "Churn"

# Get categorical columns (from full DataFrame)
categorical_cols = df.select_dtypes(include="object").columns.tolist()

# Remove target column safely
if target_col in categorical_cols:
    categorical_cols.remove(target_col)

In [14]:
# Identify categorical columns from features dataframe X, not original df
categorical_cols = X.select_dtypes(include="object").columns.tolist()

# Label encode categorical columns
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le


In [15]:
# train and test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [16]:
# Feature scaling (standardize numerical features)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [17]:
# low configuration cpu
# Define the model (simple MLP)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\Chandra\miniconda3\envs\deeplearning-customer-churn\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Train with EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=16,
    callbacks=[early_stop],
    verbose=2
)


Epoch 1/30
282/282 - 9s - 33ms/step - accuracy: 0.7888 - loss: 0.4308 - val_accuracy: 0.8199 - val_loss: 0.3700
Epoch 2/30
282/282 - 1s - 4ms/step - accuracy: 0.8460 - loss: 0.3272 - val_accuracy: 0.8305 - val_loss: 0.3413
Epoch 3/30
282/282 - 2s - 6ms/step - accuracy: 0.8520 - loss: 0.3058 - val_accuracy: 0.8429 - val_loss: 0.3300
Epoch 4/30
282/282 - 1s - 5ms/step - accuracy: 0.8600 - loss: 0.2959 - val_accuracy: 0.8509 - val_loss: 0.3255
Epoch 5/30
282/282 - 1s - 5ms/step - accuracy: 0.8644 - loss: 0.2900 - val_accuracy: 0.8447 - val_loss: 0.3194
Epoch 6/30
282/282 - 1s - 4ms/step - accuracy: 0.8653 - loss: 0.2851 - val_accuracy: 0.8447 - val_loss: 0.3264
Epoch 7/30
282/282 - 2s - 6ms/step - accuracy: 0.8689 - loss: 0.2812 - val_accuracy: 0.8465 - val_loss: 0.3219
Epoch 8/30
282/282 - 1s - 4ms/step - accuracy: 0.8695 - loss: 0.2784 - val_accuracy: 0.8492 - val_loss: 0.3214
Epoch 9/30
282/282 - 1s - 4ms/step - accuracy: 0.8715 - loss: 0.2758 - val_accuracy: 0.8527 - val_loss: 0.3230


In [19]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy:.4f}')


45/45 - 0s - 4ms/step - accuracy: 0.8573 - loss: 0.2962
Test Accuracy: 0.8573


In [20]:
# Predict and generate classification report
from sklearn.metrics import classification_report, accuracy_score

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(f'Accuracy score: {accuracy_score(y_test, y_pred):.4f}')


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1035
           1       0.75      0.70      0.72       374

    accuracy                           0.86      1409
   macro avg       0.82      0.81      0.81      1409
weighted avg       0.86      0.86      0.86      1409

Accuracy score: 0.8573


In [22]:
import os

# Step 1: Get current directory and move up one level
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Step 2: Define the new folder path where you want to save the model
saved_models = os.path.join(parent_dir, 'saved_models')

# Step 3: Create the folder if it doesn't exist
os.makedirs(saved_models, exist_ok=True)

# Step 4: Define full path for the model file
model_path = os.path.join(saved_models, 'customer_churn_model.h5')

# Step 5: Save the model
model.save(model_path)

print(f'Model saved at: {model_path}')


Model saved at: g:\DataScienceProjects\Deep-Learning-Projects\Deep-Learning-Customer-Churn\saved_models\customer_churn_model.h5
